In [2]:
# training_set = dataset_train.iloc[:, 1:3].values

# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 2)) 
# Use this to add another column to the initial matrix. When i get to the np.reshape function i used: 

# for i in range(60, 1258):
#     X_train.append(training_set_scaled[i-60:i, 0:2])

In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [4]:
dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')
trainig_set = dataset_train.iloc[:, 1:2].values

In [5]:
trainig_set

array([[325.25],
       [331.27],
       [329.83],
       ...,
       [793.7 ],
       [783.33],
       [782.75]])

In [6]:
sc = MinMaxScaler(feature_range = (0, 1))
trainig_set_scaled = sc.fit_transform(trainig_set)

In [7]:
trainig_set_scaled

array([[0.08581368],
       [0.09701243],
       [0.09433366],
       ...,
       [0.95725128],
       [0.93796041],
       [0.93688146]])

In [8]:
X_train = []
y_train = []

for i in range(60, len(trainig_set)):
    X_train.append(trainig_set_scaled[i-60:i, 0])   # i = 60 - 60 = 0 : 60 [0:59]
    y_train.append(trainig_set_scaled[i, 0]) # treinar [60] para prevê

X_train, y_train = np.array(X_train), np.array(y_train)

In [9]:
pd.DataFrame(X_train).head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.085814,0.097012,0.094334,0.091562,0.079842,0.064328,0.058542,0.065686,0.061091,0.066393,...,0.052143,0.056124,0.058189,0.065407,0.068830,0.072438,0.079935,0.078466,0.080345,0.084977
1,0.097012,0.094334,0.091562,0.079842,0.064328,0.058542,0.065686,0.061091,0.066393,0.061426,...,0.056124,0.058189,0.065407,0.068830,0.072438,0.079935,0.078466,0.080345,0.084977,0.086279
2,0.094334,0.091562,0.079842,0.064328,0.058542,0.065686,0.061091,0.066393,0.061426,0.074745,...,0.058189,0.065407,0.068830,0.072438,0.079935,0.078466,0.080345,0.084977,0.086279,0.084716
3,0.091562,0.079842,0.064328,0.058542,0.065686,0.061091,0.066393,0.061426,0.074745,0.027978,...,0.065407,0.068830,0.072438,0.079935,0.078466,0.080345,0.084977,0.086279,0.084716,0.074541
4,0.079842,0.064328,0.058542,0.065686,0.061091,0.066393,0.061426,0.074745,0.027978,0.023793,...,0.068830,0.072438,0.079935,0.078466,0.080345,0.084977,0.086279,0.084716,0.074541,0.078838


In [11]:
# (X_train_shape[0] (número de linhas)  X_train_shape[1] (número de colunas))
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [19]:
# Eixo 0 (1198): Representa o número de janelas criadas (uma para cada dia a partir do 60º). relacionado ao y_train
# Eixo 1 (60): Representa os 60 dias consecutivos dentro de cada janela.
# Eixo 2 (1): Representa o canal (único neste caso, com o valor normalizado).
for i, size in enumerate(X_train.shape):
    print(f"Eixo {i}: Tamanho = {size}")

Eixo 0: Tamanho = 1198
Eixo 1: Tamanho = 60
Eixo 2: Tamanho = 1
